<a href="https://colab.research.google.com/github/MariusMC8/Thesis-coding/blob/main/Risk_ranking_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>